# Covid-19 vs Mortality Rates By County

(Preface: I am not a medical professional and should not be taken for medical advice. I am also not authority on statistical techniques this should all be taken with a grain of salt. This notebook was created as part of a data science project through Udacity.com) 


## Introduction

I'd like to see if location and the location's mortality rates relate to Covid-19 deaths (and possibly contraction cases). A good way to tell may be to ask: can we predict the number of deaths for a location using it's mortality rates? 

Three questions will be asked:
1. Is there any county with significantly higher levels of non-covid1-9 mortality than others?
1. What causes of death correlate best with Covid-19 deaths?
1. Can we predict the current number of Covid-19 deaths based on past mortality?


### Assumptions

Not all datasets are perfect, this dataset is a combination of several sources (noted below). Our calculations are based on several assumptions.

1. Some of the data here is old (mortality rates by county in 2014)
1. Some of the data is estimated (population per county)
1. The latest date of the Covid deaths is 3-31-20. This will likely result in poor performance of our machine learning algorithm as there are few deaths so far. This is unfortunately predicted to change and the algorithms may perform better on later datasets.

### Acknowledgements

Data was collected from the following sources: 
* https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset
* https://www.kaggle.com/IHME/us-countylevel-mortality/data
* https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/

## Data Preparation

We have 2 datasets - 
1. Covid-19 deaths by county as of 3-31-20 - `us-counties.csv`
1. Mortality rates per 100,000 per county - `mort.csv`

To create one single cohesive dataset we must: 
* Manipulate and mesh the data into usable columns
* Merge relevent data in these datasets based on their FIPS code. 

#### Load the libraries

In [3]:
#Load the appropriate libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

#### Load the csv files

In [66]:
# Load datasets
covid_df = pd.read_csv('./us-counties.csv')
#pop_df = pd.read_csv('./co-est2019-alldata-1.csv',encoding = "ISO-8859-1") # encoding due to import issues
mort_df = pd.read_csv('./mort.csv')

mort_df.head() #Example

,Location,FIPS,Category,"Mortality Rate, 1980*","Mortality Rate, 1980* (Min)","Mortality Rate, 1980* (Max)","Mortality Rate, 1985*","Mortality Rate, 1985* (Min)","Mortality Rate, 1985* (Max)","Mortality Rate, 1990*",...,"Mortality Rate, 2005* (Max)","Mortality Rate, 2010*","Mortality Rate, 2010* (Min)","Mortality Rate, 2010* (Max)","Mortality Rate, 2014*","Mortality Rate, 2014* (Min)","Mortality Rate, 2014* (Max)","% Change in Mortality Rate, 1980-2014","% Change in Mortality Rate, 1980-2014 (Min)","% Change in Mortality Rate, 1980-2014 (Max)"
0,United States,NaN,Neonatal disorders,9.18,8.83,9.93,6.91,6.73,7.36,6.09,...,4.55,3.75,3.43,3.85,3.32,3.02,3.45,-63.85,-68.95,-61.55
1,Alabama,1.0,Neonatal disorders,11.03,10.57,12.00,8.51,8.25,9.12,7.52,...,6.52,5.58,5.16,5.78,5.10,4.67,5.32,-53.82,-60.08,-50.77
2,"Autauga County, Alabama",1001.0,Neonatal disorders,9.58,8.37,11.02,7.50,6.56,8.58,6.76,...,6.30,4.83,4.19,5.49,4.56,3.96,5.21,-52.43,-60.01,-46.16
3,"Baldwin County, Alabama",1003.0,Neonatal disorders,8.75,7.86,9.81,6.54,5.88,7.27,5.76,...,5.03,4.02,3.53,4.48,3.68,3.25,4.08,-57.97,-64.96,-52.87
4,"Barbour County, Alabama",1005.0,Neonatal disorders,12.30,10.64,14.15,9.26,8.13,10.60,8.12,...,7.89,5.99,5.16,6.84,5.43,4.63,6.24,-55.86,-62.49,-50.32


#### Reformat and extract necessary columns

In [41]:
# covid_df
# We only want the most up-to-date information (3-31-2020 as of this writing)
covid_df = covid_df[covid_df['date']=='2020-03-30']

# Drop columns where fips is inf/NaN
covid_df = covid_df.replace([np.inf, -np.inf], np.nan)
covid_df_clean = covid_df.dropna(subset=['fips']).copy()

# Change the fips column to int
covid_df_clean.fips = covid_df_clean.fips.astype(int)

covid_df_clean.head()

,date,county,state,fips,cases,deaths
19714,2020-03-30,Autauga,Alabama,1001,7,0
19715,2020-03-30,Baldwin,Alabama,1003,18,0
19716,2020-03-30,Bibb,Alabama,1007,2,0
19717,2020-03-30,Blount,Alabama,1009,5,0
19718,2020-03-30,Bullock,Alabama,1011,3,0


In [45]:
# pop_df - This dataset was collected because I didn't realize the mortality dataset was per 100,000 already.
# We only want the fips code (we'll need to create new column) and the estimated population for 2014
#pop_df['fips'] = (pop_df['STATE'].astype(str) + pop_df['COUNTY'].apply(lambda x: str(x).zfill(3))).astype(int)
#pop_df_clean = pop_df[['fips','POPESTIMATE2014']]
#pop_df_clean.head()

In [69]:
# mort_df
# Let's start by only grabbing the relevent data (fips and mortality rate for most recent date (2014))
mort_df_temp = mort_df[['FIPS','Category','Mortality Rate, 2014*']]

In [68]:
# Perform a pivot - the data is ugly as it is and clean up na values
mort_df_clean = mort_df_temp.pivot(index='FIPS', columns='Category', values='Mortality Rate, 2014*').reset_index()
mort_df_clean.dropna(inplace=True)
mort_df_clean.FIPS = mort_df_clean.FIPS.astype(int)

mort_df_clean

Category,FIPS,Cardiovascular diseases,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,"Diabetes, urogenital, blood, and endocrine diseases","Diarrhea, lower respiratory, and other common infectious diseases",Digestive diseases,"Forces of nature, war, and legal intervention",HIV/AIDS and tuberculosis,Maternal disorders,...,Neglected tropical diseases and malaria,Neonatal disorders,Neoplasms,Neurological disorders,Nutritional deficiencies,"Other communicable, maternal, neonatal, and nutritional diseases",Other non-communicable diseases,Self-harm and interpersonal violence,Transport injuries,Unintentional injuries
1,1,310.59,73.44,21.76,70.81,40.77,17.13,0.10,2.94,0.40,...,0.05,5.10,230.98,119.14,1.99,1.49,7.60,26.53,24.21,23.02
2,2,217.01,51.36,20.17,49.17,19.95,15.05,0.10,1.27,0.19,...,0.07,2.35,194.76,101.06,1.64,1.41,6.74,31.54,15.87,25.09
3,4,209.33,51.93,20.94,50.22,21.39,12.53,0.05,1.82,0.23,...,0.07,2.71,170.27,90.23,0.94,1.77,5.64,25.93,15.75,21.50
4,5,324.15,69.63,18.94,66.73,41.57,16.96,0.16,2.10,0.50,...,0.08,3.88,219.87,100.74,2.27,1.48,7.78,27.51,23.75,23.11
5,6,221.94,41.94,16.87,46.94,23.48,11.85,0.04,2.47,0.24,...,0.05,2.27,165.36,79.58,0.66,1.47,4.46,16.12,10.64,11.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56037,262.99,78.01,23.48,50.38,41.89,19.64,0.09,0.41,0.37,...,0.11,3.40,174.19,96.92,1.59,1.02,6.28,30.28,26.39,28.23
3190,56039,144.71,36.09,10.15,27.66,22.32,14.76,0.24,0.35,0.28,...,0.09,2.20,127.30,83.65,1.34,0.83,4.21,16.55,15.98,20.76
3191,56041,266.75,81.51,20.30,55.84,32.23,22.31,0.16,0.40,0.44,...,0.09,3.38,183.72,88.20,3.00,1.03,7.72,30.15,24.37,31.71
3192,56043,227.19,59.36,18.27,54.20,45.79,19.06,0.12,0.37,0.37,...,0.17,2.78,182.46,82.74,1.44,1.03,6.08,22.43,24.50,25.45
